In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import warnings
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from sklearn.metrics import classification_report
from random import shuffle

C:\Users\nataw\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nataw\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nataw\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nataw\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [40]:
from sklearn import metrics

In [2]:
def load_images_from_folder():
    img_size = 64
    images = []
    image_paths = 'E:/Drowsiness/Data_CNN/train/alert/'
    for filename in os.listdir(image_paths):
        img = cv2.imread(os.path.join(image_paths,filename))
        if img is not None:
            img = cv2.resize(img,(img_size,img_size))
            images.append([np.array(img),[1,0]])
    
    image_paths = 'E:/Drowsiness/Data_CNN/train/drowsy/'
    for filename in os.listdir(image_paths):
        img = cv2.imread(os.path.join(image_paths,filename))
        if img is not None:
            img = cv2.resize(img,(img_size,img_size))
            images.append([np.array(img),[0,1]])
    shuffle(images)
    return images

In [3]:
def load_images_from_folder_test():
    img_size = 64
    images = []
    image_paths = 'E:/Drowsiness/Data_CNN/test/alert/'
    for filename in os.listdir(image_paths):
        img = cv2.imread(os.path.join(image_paths,filename))
        if img is not None:
            img = cv2.resize(img,(img_size,img_size))
            images.append([np.array(img),[1,0]])
    
    image_paths = 'E:/Drowsiness/Data_CNN/test/drowsy/'
    for filename in os.listdir(image_paths):
        img = cv2.imread(os.path.join(image_paths,filename))
        if img is not None:
            img = cv2.resize(img,(img_size,img_size))
            images.append([np.array(img),[0,1]])
    shuffle(images)
    return images

In [4]:
train = load_images_from_folder()

In [5]:
train = np.array(train)

In [6]:
train.shape

(4851, 2)

In [8]:
X_train = np.array([i[0] for i in train]).reshape(-1,64,64,3)/255.

In [9]:
y_train = np.array([i[1] for i in train])

In [10]:
train = 0

In [11]:
datagen = ImageDataGenerator(rotation_range=30,width_shift_range=0.2)
datagen.fit(X_train)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [43]:
from keras import backend
from keras.models import Sequential
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.layers.core import Dense, Flatten,Activation,Dropout
model = Sequential()
# Layer 1: Convolutional Layer
model.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(64,64,3), activation='relu',))
# Layer 2: Pooling Layer
model.add(MaxPooling2D(pool_size=(3,3)))
# Layer 3: Convolutional Layer
model.add(Conv2D(filters=32, kernel_size=(3,3), input_shape=(64,64,3), activation='relu',))
# Layer 4: Pooling Layer
model.add(MaxPooling2D(pool_size=(3,3)))
# Layer 5: Flatten Layer
model.add(Conv2D(filters=64, kernel_size=(3,3), input_shape=(64,64,3), activation='relu',))

model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Flatten())
# Layer 6: Dense Layer (Hidden Layer)
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
# Layer 7: Dense Layer (Output Layer)
model.add(Dense(2, activation='softmax'))

In [44]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 18, 18, 32)        9248      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 4, 4, 64)          18496     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 1, 1, 64)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 64)               

In [45]:
model.compile(optimizer="adam",loss='categorical_crossentropy',metrics=['accuracy'])

In [46]:
tf.debugging.set_log_device_placement(True)
model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_test,y_test),
                    steps_per_epoch=len(X_train) / 32, epochs=50)

Epoch 1/50
122/121 [==============================] - 4s 32ms/step - loss: 0.6648 - accuracy: 0.6152 - val_loss: 0.6746 - val_accuracy: 0.6437
Epoch 2/50
122/121 [==============================] - 4s 30ms/step - loss: 0.6456 - accuracy: 0.6371 - val_loss: 0.6247 - val_accuracy: 0.6498
Epoch 3/50
122/121 [==============================] - 4s 31ms/step - loss: 0.6392 - accuracy: 0.6443 - val_loss: 0.6175 - val_accuracy: 0.6612
Epoch 4/50
122/121 [==============================] - 4s 31ms/step - loss: 0.6308 - accuracy: 0.6389 - val_loss: 0.6097 - val_accuracy: 0.6571
Epoch 5/50
122/121 [==============================] - 4s 31ms/step - loss: 0.6153 - accuracy: 0.6585 - val_loss: 0.6042 - val_accuracy: 0.6735
Epoch 6/50
122/121 [==============================] - 4s 32ms/step - loss: 0.5936 - accuracy: 0.6874 - val_loss: 0.5449 - val_accuracy: 0.7178
Epoch 7/50
122/121 [==============================] - 4s 31ms/step - loss: 0.5775 - accuracy: 0.6951 - val_loss: 0.5247 - val_accuracy: 0.7343

.

In [50]:
model.save('CNN_64_50.h5')

<h1>เพิ่ม 50 รอบ<h1>

In [51]:
model = load_model('CNN_64_50.h5')
tf.debugging.set_log_device_placement(True)
model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_test,y_test),
                    steps_per_epoch=len(X_train) / 32, epochs=50)

Epoch 1/50
122/121 [==============================] - 4s 33ms/step - loss: 0.1514 - accuracy: 0.9361 - val_loss: 0.2062 - val_accuracy: 0.9197
Epoch 2/50
122/121 [==============================] - 4s 30ms/step - loss: 0.1549 - accuracy: 0.9446 - val_loss: 0.1895 - val_accuracy: 0.9310
Epoch 3/50
122/121 [==============================] - 4s 30ms/step - loss: 0.1494 - accuracy: 0.9420 - val_loss: 0.2694 - val_accuracy: 0.9042
Epoch 4/50
122/121 [==============================] - 4s 31ms/step - loss: 0.1680 - accuracy: 0.9338 - val_loss: 0.1821 - val_accuracy: 0.9248
Epoch 5/50
122/121 [==============================] - 4s 30ms/step - loss: 0.1488 - accuracy: 0.9451 - val_loss: 0.1821 - val_accuracy: 0.9372
Epoch 6/50
122/121 [==============================] - 4s 30ms/step - loss: 0.1580 - accuracy: 0.9369 - val_loss: 0.2186 - val_accuracy: 0.9228
Epoch 7/50
122/121 [==============================] - 4s 32ms/step - loss: 0.1533 - accuracy: 0.9461 - val_loss: 0.1974 - val_accuracy: 0.9310

In [53]:
model.save('CNN_64_100.h5')

<h1>เพิ่ม 50 รอบ<h1>

In [54]:
model = load_model('CNN_64_100.h5')
tf.debugging.set_log_device_placement(True)
model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_test,y_test),
                    steps_per_epoch=len(X_train) / 32, epochs=50)

Epoch 1/50
122/121 [==============================] - 4s 32ms/step - loss: 0.1238 - accuracy: 0.9521 - val_loss: 0.1790 - val_accuracy: 0.9320
Epoch 2/50
122/121 [==============================] - 4s 30ms/step - loss: 0.1173 - accuracy: 0.9541 - val_loss: 0.2109 - val_accuracy: 0.9238
Epoch 3/50
122/121 [==============================] - 4s 31ms/step - loss: 0.1093 - accuracy: 0.9590 - val_loss: 0.1698 - val_accuracy: 0.9320
Epoch 4/50
122/121 [==============================] - 4s 32ms/step - loss: 0.1125 - accuracy: 0.9575 - val_loss: 0.1586 - val_accuracy: 0.9413
Epoch 5/50
122/121 [==============================] - 4s 31ms/step - loss: 0.1138 - accuracy: 0.9575 - val_loss: 0.1754 - val_accuracy: 0.9310
Epoch 6/50
122/121 [==============================] - 4s 32ms/step - loss: 0.1116 - accuracy: 0.9541 - val_loss: 0.1647 - val_accuracy: 0.9434
Epoch 7/50
122/121 [==============================] - 4s 30ms/step - loss: 0.1132 - accuracy: 0.9562 - val_loss: 0.1487 - val_accuracy: 0.9495

In [55]:
model.save('CNN_64_150.h5')

50

In [81]:
model = load_model('CNN_64_150.h5')
tf.debugging.set_log_device_placement(True)
model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_test,y_test),
                    steps_per_epoch=len(X_train) / 32, epochs=50)

Epoch 1/50
122/121 [==============================] - 4s 32ms/step - loss: 0.0979 - accuracy: 0.9608 - val_loss: 0.1620 - val_accuracy: 0.9485
Epoch 2/50
122/121 [==============================] - 4s 30ms/step - loss: 0.1007 - accuracy: 0.9626 - val_loss: 0.1644 - val_accuracy: 0.9454
Epoch 3/50
122/121 [==============================] - 4s 31ms/step - loss: 0.0977 - accuracy: 0.9642 - val_loss: 0.1463 - val_accuracy: 0.9475
Epoch 4/50
122/121 [==============================] - 4s 30ms/step - loss: 0.0948 - accuracy: 0.9629 - val_loss: 0.1620 - val_accuracy: 0.9516
Epoch 5/50
122/121 [==============================] - 4s 31ms/step - loss: 0.1018 - accuracy: 0.9637 - val_loss: 0.1903 - val_accuracy: 0.9320
Epoch 6/50
122/121 [==============================] - 4s 31ms/step - loss: 0.0962 - accuracy: 0.9626 - val_loss: 0.1880 - val_accuracy: 0.9372
Epoch 7/50
122/121 [==============================] - 4s 30ms/step - loss: 0.0862 - accuracy: 0.9670 - val_loss: 0.1754 - val_accuracy: 0.9413

In [82]:
model.save('CNN_64_200.h5')

100

In [87]:
model = load_model('CNN_64_200.h5')
tf.debugging.set_log_device_placement(True)
model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_test,y_test),
                    steps_per_epoch=len(X_train) / 32, epochs=100)

Epoch 1/100
122/121 [==============================] - 4s 32ms/step - loss: 0.0800 - accuracy: 0.9706 - val_loss: 0.1766 - val_accuracy: 0.9403
Epoch 2/100
122/121 [==============================] - 4s 30ms/step - loss: 0.0955 - accuracy: 0.9621 - val_loss: 0.1795 - val_accuracy: 0.9361
Epoch 3/100
122/121 [==============================] - 4s 32ms/step - loss: 0.0807 - accuracy: 0.9670 - val_loss: 0.1692 - val_accuracy: 0.9444
Epoch 4/100
122/121 [==============================] - 4s 31ms/step - loss: 0.0854 - accuracy: 0.9662 - val_loss: 0.1607 - val_accuracy: 0.9423
Epoch 5/100
122/121 [==============================] - 4s 31ms/step - loss: 0.0882 - accuracy: 0.9665 - val_loss: 0.1671 - val_accuracy: 0.9403
Epoch 6/100
122/121 [==============================] - 4s 32ms/step - loss: 0.0821 - accuracy: 0.9670 - val_loss: 0.1817 - val_accuracy: 0.9434
Epoch 7/100
122/121 [==============================] - 4s 31ms/step - loss: 0.0913 - accuracy: 0.9629 - val_loss: 0.1452 - val_accuracy:

122/121 [==============================] - 4s 34ms/step - loss: 0.0672 - accuracy: 0.9735 - val_loss: 0.2059 - val_accuracy: 0.9331
Epoch 58/100
122/121 [==============================] - 4s 33ms/step - loss: 0.0778 - accuracy: 0.9727 - val_loss: 0.1502 - val_accuracy: 0.9557
Epoch 59/100
122/121 [==============================] - 4s 33ms/step - loss: 0.0727 - accuracy: 0.9729 - val_loss: 0.1600 - val_accuracy: 0.9547
Epoch 60/100
122/121 [==============================] - 4s 34ms/step - loss: 0.0669 - accuracy: 0.9740 - val_loss: 0.1820 - val_accuracy: 0.9516
Epoch 61/100
122/121 [==============================] - 4s 34ms/step - loss: 0.0666 - accuracy: 0.9719 - val_loss: 0.1456 - val_accuracy: 0.9578
Epoch 62/100
122/121 [==============================] - 4s 35ms/step - loss: 0.0646 - accuracy: 0.9771 - val_loss: 0.1739 - val_accuracy: 0.9495
Epoch 63/100
122/121 [==============================] - 5s 41ms/step - loss: 0.0790 - accuracy: 0.9680 - val_loss: 0.1694 - val_accuracy: 0.952

In [88]:
model.save('CNN_64_300.h5')

<h1>ทดสอบโมเดล<h1>

In [56]:
test = load_images_from_folder_test()

In [57]:
X_testt = np.array([i[0] for i in test]).reshape(-1,64,64,3)/255.

In [58]:
y_testt = np.array([i[1] for i in test])

<h2>โมดล50รอบ<h2>

In [59]:
rounded_labels=np.argmax(y_testt, axis=1)
rounded_labels[1]

0

In [60]:
model = load_model('CNN_64_50.h5')

In [61]:
model.evaluate(X_testt, y_testt, verbose=0)

[0.17052625681994096, 0.946540892124176]

In [62]:
prediction = model.predict_classes(X_testt)
print(classification_report(rounded_labels, prediction))

              precision    recall  f1-score   support

           0       0.98      0.91      0.95       338
           1       0.91      0.98      0.95       298

    accuracy                           0.95       636
   macro avg       0.95      0.95      0.95       636
weighted avg       0.95      0.95      0.95       636



In [63]:
prediction = model.predict_classes(X_testt)
confusion_matrix(rounded_labels, prediction)

array([[309,  29],
       [  5, 293]], dtype=int64)

<h2>โมดล100รอบ<h2>

In [64]:
model = load_model('CNN_64_100.h5')

In [65]:
model.evaluate(X_testt, y_testt, verbose=0)

[0.24036825902806888, 0.9308176040649414]

In [66]:
prediction = model.predict_classes(X_testt)
print(classification_report(rounded_labels, prediction))

              precision    recall  f1-score   support

           0       0.98      0.88      0.93       338
           1       0.88      0.98      0.93       298

    accuracy                           0.93       636
   macro avg       0.93      0.93      0.93       636
weighted avg       0.94      0.93      0.93       636



In [67]:
prediction = model.predict_classes(X_testt)
confusion_matrix(rounded_labels, prediction)

array([[299,  39],
       [  5, 293]], dtype=int64)

<h2>โมดล150รอบ<h2>

In [72]:
model = load_model('CNN_64_150.h5')

In [73]:
model.evaluate(X_testt, y_testt, verbose=0)

[0.21937202516966645, 0.9355345964431763]

In [74]:
prediction = model.predict_classes(X_testt)
print(classification_report(rounded_labels, prediction))

              precision    recall  f1-score   support

           0       0.97      0.90      0.94       338
           1       0.90      0.97      0.93       298

    accuracy                           0.94       636
   macro avg       0.94      0.94      0.94       636
weighted avg       0.94      0.94      0.94       636



In [75]:
prediction = model.predict_classes(X_testt)
confusion_matrix(rounded_labels, prediction)

array([[305,  33],
       [  8, 290]], dtype=int64)

In [83]:
model = load_model('CNN_64_200.h5')

In [84]:
model.evaluate(X_testt, y_testt, verbose=0)

[0.28855671897624274, 0.9308176040649414]

In [85]:
prediction = model.predict_classes(X_testt)
print(classification_report(rounded_labels, prediction))

              precision    recall  f1-score   support

           0       0.97      0.89      0.93       338
           1       0.89      0.97      0.93       298

    accuracy                           0.93       636
   macro avg       0.93      0.93      0.93       636
weighted avg       0.93      0.93      0.93       636



In [86]:
prediction = model.predict_classes(X_testt)
confusion_matrix(rounded_labels, prediction)

array([[302,  36],
       [  8, 290]], dtype=int64)

In [89]:
model = load_model('CNN_64_300.h5')

In [90]:
model.evaluate(X_testt, y_testt, verbose=0)

[0.16328314890786302, 0.9496855139732361]

In [91]:
prediction = model.predict_classes(X_testt)
print(classification_report(rounded_labels, prediction))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95       338
           1       0.94      0.95      0.95       298

    accuracy                           0.95       636
   macro avg       0.95      0.95      0.95       636
weighted avg       0.95      0.95      0.95       636



In [92]:
prediction = model.predict_classes(X_testt)
confusion_matrix(rounded_labels, prediction)

array([[321,  17],
       [ 15, 283]], dtype=int64)